## Op Titanic Simple Sample

Derived from https://github.com/salesforce/TransmogrifAI/tree/master/helloworld/notebooks

Here we describe a very simple TransmogrifAI workflow for predicting survivors in the often-cited Titanic dataset. The code for building and applying the Titanic model can be found here: Titanic Code, and the data can be found here: [Titanic Data](https://github.com/salesforce/op/blob/master/helloworld/src/main/resources/TitanicDataset/TitanicPassengersTrainData.csv).

First we need to load transmogrifai and Spark Mllib jars


In [1]:
import $ivy.`org.apache.spark::spark-sql:2.3.3`
import $ivy.`org.apache.spark::spark-mllib:2.3.3`
import $ivy.`sh.almond::almond-spark:0.3.1`
import $ivy.`com.salesforce.transmogrifai::transmogrifai-core:0.5.1`

import $ivy.$                                  

import $ivy.$                                    

import $ivy.$                              

import $ivy.$                                                       

Import relevant classes

In [2]:
import com.salesforce.op._
import com.salesforce.op.features._
import com.salesforce.op.features.types._
import com.salesforce.op.evaluators.Evaluators

import com.salesforce.op._

import com.salesforce.op.features._

import com.salesforce.op.features.types._

import com.salesforce.op.evaluators.Evaluators

In [3]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}


Instantiate a SparkSession

In [4]:
import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .appName("TitanicPrediction")
    .getOrCreate()
}

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@552cdcdc

Let us create a case class to describe the schema for the data:

In [5]:
import spark.implicits._

// Needed for now for case classes defined withing Ammonite. Won't be necessary in future versions of Spark.
// See https://github.com/alexarchambault/ammonite-spark/issues/19 and https://github.com/apache/spark/pull/23607
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this);
case class Passenger(
  id: Int,
  survived: Int,
  pClass: Option[Int],
  name: Option[String],
  sex: Option[String],
  age: Option[Double],
  sibSp: Option[Int],
  parCh: Option[Int],
  ticket: Option[String],
  fare: Option[Double],
  cabin: Option[String],
  embarked: Option[String]
)

import spark.implicits._

// Needed for now for case classes defined withing Ammonite. Won't be necessary in future versions of Spark.
// See https://github.com/alexarchambault/ammonite-spark/issues/19 and https://github.com/apache/spark/pull/23607

defined class Passenger

We then define the set of raw features that we would like to extract from the data. The raw features are defined using [FeatureBuilders](https://docs.transmogrif.ai/Developer-Guide#featurebuilders), and are strongly typed. TransmogrifAI supports the following basic feature types: `Text`, `Numeric`, `Vector`, `List` , `Set`, `Map`. 
In addition it supports many specific feature types which extend these base types: Email extends Text; Integral, Real and Binary extend Numeric; Currency and Percentage extend Real. For a complete view of the types supported see the Type Hierarchy and Automatic Feature Engineering section in the Documentation.

Basic `FeatureBuilders` will be created for you if you use the TransmogrifAI CLI to bootstrap your project as described here. However, it is often useful to edit this code to customize feature generation and take full advantage of the Feature types available (selecting the appropriate type will improve automatic feature engineering steps).

When defining raw features, specify the extract logic to be applied to the raw data, and also annotate the features as either predictor or response variables via the FeatureBuilders:

In [6]:
val survived = FeatureBuilder.RealNN[Passenger].extract(_.survived.toRealNN).asResponse
val pClass = FeatureBuilder.PickList[Passenger].extract(_.pClass.map(_.toString).toPickList).asPredictor
val name = FeatureBuilder.Text[Passenger].extract(_.name.toText).asPredictor
val sex = FeatureBuilder.PickList[Passenger].extract(_.sex.map(_.toString).toPickList).asPredictor
val age = FeatureBuilder.Real[Passenger].extract(_.age.toReal).asPredictor
val sibSp = FeatureBuilder.Integral[Passenger].extract(_.sibSp.toIntegral).asPredictor
val parCh = FeatureBuilder.Integral[Passenger].extract(_.parCh.toIntegral).asPredictor
val ticket = FeatureBuilder.PickList[Passenger].extract(_.ticket.map(_.toString).toPickList).asPredictor
val fare = FeatureBuilder.Real[Passenger].extract(_.fare.toReal).asPredictor
val cabin = FeatureBuilder.PickList[Passenger].extract(_.cabin.map(_.toString).toPickList).asPredictor
val embarked = FeatureBuilder.PickList[Passenger].extract(_.embarked.map(_.toString).toPickList).asPredictor

survived: Feature[RealNN] = Feature(
  "survived",
  true,
  FeatureGeneratorStage_000000000001,
  List(),
  "RealNN_000000000001",
  List()
)
pClass: Feature[PickList] = Feature(
  "pClass",
  false,
  FeatureGeneratorStage_000000000002,
  List(),
  "PickList_000000000002",
  List()
)
name: Feature[Text] = Feature(
  "name",
  false,
  FeatureGeneratorStage_000000000003,
  List(),
  "Text_000000000003",
  List()
)
sex: Feature[PickList] = Feature(
  "sex",
  false,
  FeatureGeneratorStage_000000000004,
  List(),
  "PickList_000000000004",
  List()
)
age: Feature[Real] = Feature(
  "age",
  false,
  FeatureGeneratorStage_000000000005,
  List(),
  "Real_000000000005",
  List()
)
sibSp: Feature[Integral] = Feature(
  "sibSp",
  false,
  FeatureGeneratorStage_000000000006,
  List(),
  "Integral_000000000006",
  List()
)
parCh: Feature[Integral] = Feature(
  "parCh",
  false,
  FeatureGeneratorStage_000000000007,
  List(),
  "Integral_000000000007",
  List()
)
ticket: Feature[PickList] = F

Now that the raw features have been defined, we go ahead and define how we would like to manipulate them via Stages (Transformers and Estimators). A TransmogrifAI Stage takes one or more Features, and returns a new Feature. TransmogrifAI provides numerous handy short cuts for specifying common feature manipulations. For basic arithmetic operations, you can just use “+”, “-“, “*” and “/”. In addition, shortcuts like “normalize”, “pivot” and “map” are also available.

In [7]:
val familySize = sibSp + parCh + 1
val estimatedCostOfTickets = familySize * fare
val pivotedSex = sex.pivot()
val normedAge = age.fillMissingWithMean().zNormalize()
val ageGroup = age.map[PickList](_.value.map(v => if (v > 18) "adult" else "child").toPickList)

familySize: FeatureLike[Real] = Feature(
  "parCh-sibSp_2-stagesApplied_Real_00000000000d",
  false,
  UnaryLambdaTransformer_00000000000d,
  WrappedArray(
    Feature(
      "parCh-sibSp_1-stagesApplied_Real_00000000000c",
      false,
      BinaryLambdaTransformer_00000000000c,
      WrappedArray(
        Feature(
          "sibSp",
          false,
          FeatureGeneratorStage_000000000006,
          List(),
          "Integral_000000000006",
          List()
        ),
        Feature(
          "parCh",
          false,
          FeatureGeneratorStage_000000000007,
          List(),
          "Integral_000000000007",
          List()
        )
      ),
      "Real_00000000000c",
      List()
    )
  ),
  "Real_00000000000d",
  List()
)
estimatedCostOfTickets: FeatureLike[Real] = Feature(
  "fare-parCh-sibSp_3-stagesApplied_Real_00000000000e",
  false,
  BinaryLambdaTransformer_00000000000e,
  WrappedArray(
    Feature(
      "parCh-sibSp_2-stagesApplied_Real_00000000000d",
    

See [Creating Shortcuts for Transformers and Estimators](https://docs.transmogrif.ai/en/stable/developer-guide#creating-shortcuts-for-transformers-and-estimators) for more documentation on how shortcuts for stages can be created. We now define a Feature of type `Vector`, that is a vector representation of all the features we would like to use as predictors in our workflow.

In [8]:
val passengerFeatures = Seq(
  pClass, name, age, sibSp, parCh, ticket,
  cabin, embarked, familySize, estimatedCostOfTickets,
  pivotedSex, ageGroup
).transmogrify()

passengerFeatures: FeatureLike[OPVector] = Feature(
  "age-cabin-embarked-fare-name-pClass-parCh-sex-sibSp-ticket_10-stagesApplied_OPVector_000000000017",
  false,
  VectorsCombiner_000000000017,
  WrappedArray(
    Feature(
      "parCh-sibSp_1-stagesApplied_OPVector_000000000013",
      false,
      IntegralVectorizer_000000000013,
      WrappedArray(
        Feature(
          "sibSp",
          false,
          FeatureGeneratorStage_000000000006,
          List(),
          "Integral_000000000006",
          List()
        ),
        Feature(
          "parCh",
          false,
          FeatureGeneratorStage_000000000007,
          List(),
          "Integral_000000000007",
          List()
        )
      ),
      "OPVector_000000000013",
      List()
    ),
    Feature(
      "sex_1-stagesApplied_OPVector_00000000000f",
      false,
      OpSetVectorizer_00000000000f,
      WrappedArray(
        Feature(
          "sex",
          false,
...

The `.transmogrify()` shortcut is a special AutoML Estimator that applies a default set of transformations to all the specified inputs and combines them into a single vector. This is in essence the automatic feature engineering Stage of TransmogrifAI. This stage can be discarded in favor of hand-tuned feature engineering and manual vector creation followed by combination using the `VectorsCombiner` `Transformer` (short-hand `Seq(....).combine()`) if the user desires to have complete control over feature engineering.

The next stage applies another powerful AutoML Estimator — the `SanityChecker`. The `SanityChecker` applies a variety of statistical tests to the data based on Feature types and discards predictors that are indicative of label leakage or that show little to no predictive power. This is in essence the automatic feature selection Stage of TransmogrifAI:

In [9]:
val sanityCheck = true
val finalFeatures = if (sanityCheck) survived.sanityCheck(passengerFeatures) else passengerFeatures

sanityCheck: Boolean = true
finalFeatures: FeatureLike[OPVector] = Feature(
  "age-cabin-embarked-fare-name-pClass-parCh-sex-sibSp-survived-ticket_11-stagesApplied_OPVector_000000000018",
  false,
  SanityChecker_000000000018,
  WrappedArray(
    Feature(
      "survived",
      true,
      FeatureGeneratorStage_000000000001,
      List(),
      "RealNN_000000000001",
      List()
    ),
    Feature(
      "age-cabin-embarked-fare-name-pClass-parCh-sex-sibSp-ticket_10-stagesApplied_OPVector_000000000017",
      false,
      VectorsCombiner_000000000017,
      WrappedArray(
        Feature(
          "parCh-sibSp_1-stagesApplied_OPVector_000000000013",
          false,
          IntegralVectorizer_000000000013,
          WrappedArray(
            Feature(
              "sibSp",
              false,
              FeatureGeneratorStage_000000000006,
              List(),
              "Integral_000000000006",
              List()
            ),
            Feature(
              "parCh",


Finally, the `OpLogisticRegression` Estimator is applied to derive a new triplet of Features which are essentially probabilities and predictions returned by the logistic regression algorithm:

In [10]:
import com.salesforce.op.stages.impl.classification.BinaryClassificationModelSelector
import com.salesforce.op.stages.impl.classification.BinaryClassificationModelsToTry._

val prediction = BinaryClassificationModelSelector.withTrainValidationSplit(
  modelTypesToUse = Seq(OpLogisticRegression)
).setInput(survived, finalFeatures).getOutput()

import com.salesforce.op.stages.impl.classification.BinaryClassificationModelSelector

import com.salesforce.op.stages.impl.classification.BinaryClassificationModelsToTry._


prediction: FeatureLike[Prediction] = Feature(
  "age-cabin-embarked-fare-name-pClass-parCh-sex-sibSp-survived-ticket_12-stagesApplied_Prediction_000000000023",
  true,
  ModelSelector_000000000023,
  WrappedArray(
    Feature(
      "survived",
      true,
      FeatureGeneratorStage_000000000001,
      List(),
      "RealNN_000000000001",
      List()
    ),
    Feature(
      "age-cabin-embarked-fare-name-pClass-parCh-sex-sibSp-survived-ticket_11-stagesApplied_OPVector_000000000018",
      false,
      SanityChecker_000000000018,
      WrappedArray(
        Feature(
          "survived",
          true,
          FeatureGeneratorStage_000000000001,
          List(),
          "RealNN_000000000001",
          List()
        ),
        Feature(
          "age-cabin-embarked-fare-name-pClass-parCh-sex-sibSp-ticket

In [11]:
val evaluator = Evaluators.BinaryClassification().setLabelCol(survived).setPredictionCol(prediction)

evaluator: evaluators.OpBinaryClassificationEvaluator = OpBinaryClassificationEvaluator_000000000024

Notice that everything we’ve done so far has been purely at the level of definitions. We have defined how we would like to extract our raw features from data of type `Passenger`, and we have defined how we would like to manipulate them. In order to actually manifest the data described by these features, we need to add them to a workflow and attach a data source to the workflow.

Please note the `trainFilePath` is the derived path from folder where host folder is mounted as a volume (/home/beakerx/helloworld) in this case. This can be changed as well depending on the location and volume director you are mounting the data from. You can also create a new DataReader with a new path.

In [12]:
import spark.implicits._ // Needed for Encoders for the Passenger case class
import com.salesforce.op.readers.DataReaders

val trainFilePath = "datasets/TitanicDataset/TitanicPassengersTrainData.csv"
// Define a way to read data into our Passenger class from our CSV file
val trainDataReader = DataReaders.Simple.csvCase[Passenger](
  path = Option(trainFilePath),
  key = _.id.toString
)

import spark.implicits._ // Needed for Encoders for the Passenger case class

import com.salesforce.op.readers.DataReaders


trainFilePath: String = "datasets/TitanicDataset/TitanicPassengersTrainData.csv"
trainDataReader: readers.CSVProductReader[Passenger] = com.salesforce.op.readers.CSVProductReader@359b1b6a

In [13]:
val workflow = new OpWorkflow()
  .setResultFeatures(survived, prediction)
  .setReader(trainDataReader)

: 

When we now call `train` on this workflow, it automatically computes and executes the entire DAG of Stages needed to compute the features survived, prediction, rawPrediction, and prob, fitting all the estimators on the training data in the process. Calling score on the fitted workflow then transforms the underlying training data to produce a DataFrame with the all the features manifested. The score method can optionally be passed an evaluator that produces metrics.

In [ ]:
val fittedWorkflow = workflow.train()
println("Summary:\n" + fittedWorkflow.summaryPretty())

After model has been fitted we use `scoreAndEvaluate()` function to evaluate the metrics

In [ ]:
println("Scoring the model:\n=================")
val (dataframe, metrics) = fittedWorkflow.scoreAndEvaluate(evaluator = evaluator)

println("Transformed dataframe columns:\n--------------------------")
dataframe.columns.foreach(println)

println("Metrics:\n------------")
println(metrics)